In [1]:
import numpy as np
import pandas as pd
import praw
import re
from datetime import datetime
from fake_useragent import UserAgent
from scipy.stats import randint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import LinearSVC
#from sklearn.naive_bayes import MultinomialNB
#from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
def no_num(string):
    """Removes all the numbers from a string"""
    r = re.sub('(\d)+', '', string.lower())
    return r

In [3]:
def reddit_news_fech(data_path, track_path):
    ua = UserAgent()

    reddit = praw.Reddit(client_id='63Stmt93yKE2Zw', 
                         client_secret='YQ-z1FWG3iQ-hmxJTzbjYveWAh4', 
                         user_agent=ua.firefox)

    if reddit.read_only == True:
        print 'We good to Go!'
    else:
        print 'Why no work?'
        return None

    data = []

    for sub in reddit.subreddit('news').top(time_filter='day', limit=100):
    
        rn = pd.to_datetime(datetime.utcnow())
    
        data.append([sub.created_utc, rn, sub.title,
                     sub.author, sub.domain, sub.ups])

    print 'Done feching data'

    df = pd.DataFrame(data, columns=['created_utc', 'scraped_utc', 'title',
                                     'author', 'domain', 'upvotes'])

    df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
    df['up_time'] = df['scraped_utc'] - df['created_utc']
    df['up_seconds'] = [x.total_seconds() for x in df['up_time']]

    median = df['upvotes'].median()
    mean = df['upvotes'].mean()
    tracked = pd.read_csv(track_path)
    track = pd.DataFrame([[rn.day+(rn.hour/24.0), median]],
                         columns=['time', 'median'])
    tracked = tracked.append(track)

    df['target'] = [1 if x > median else 0 for x in df['upvotes']]

    print 'Writing data to .csv'
    data_path = '{}/{}-{}_{}.csv'.format(data_path, rn.month, rn.day, rn.hour)
    df.to_csv(data_path, index=False, encoding='utf-8')
    tracked.to_csv(track_path, index=False)
    del tracked
    del df
    print 'Job Done'
    return data_path

In [4]:
def reddit_data_process(train, test):
    """For making data"""

    y_train = train['target']
    y_test = test['target']
    
    y_train_alt = [1 if x > train['upvotes'].mean() else 0 for x in train['upvotes']]
    y_test_alt = [1 if x > test['upvotes'].mean() else 0 for x in test['upvotes']]

    # The author is sometimes deleated, so i need to account for that
    author_train = ['REDACTED' if x == None or '' else x if isinstance(x, basestring) == True else x.name for x in train['author']]
    author_test = ['REDACTED' if x == None or '' else x if isinstance(x, basestring) == True else x.name for x in test['author']]
    cv_a = CountVectorizer(lowercase=False, token_pattern='\S+')
    cv_a.fit(author_train)
    author_train = pd.DataFrame(cv_a.transform(author_train).todense(),
                                columns=cv_a.get_feature_names())
    author_test = pd.DataFrame(cv_a.transform(author_test).todense(),
                               columns=cv_a.get_feature_names())
    del cv_a

    cv_d = CountVectorizer(token_pattern='\S+')
    cv_d.fit(train['domain'])
    domain_train = pd.DataFrame(cv_d.transform(train['domain']).todense(),
                                columns=cv_d.get_feature_names())
    domain_test = pd.DataFrame(cv_d.transform(test['domain']).todense(),
                               columns=cv_d.get_feature_names())
    del cv_d
    
    cv_t = CountVectorizer(stop_words='english', preprocessor=no_num)
    cv_t.fit(train['title'])
    title_train = pd.DataFrame(cv_t.transform(train['title']).todense(),
                               columns=cv_t.get_feature_names())
    title_test = pd.DataFrame(cv_t.transform(test['title']).todense(),
                              columns=cv_t.get_feature_names())
    del cv_t
    
    ad_train = author_train.merge(domain_train, left_index=True, right_index=True)
    super_train = title_train.merge(ad_train, left_index=True, right_index=True)
    ad_test = author_test.merge(domain_test, left_index=True, right_index=True)
    super_test = title_test.merge(ad_test, left_index=True, right_index=True)

    super_train['up_seconds'] = train['up_seconds'].values
    super_test['up_seconds'] = test['up_seconds'].values
    
    del author_train
    del author_test
    del domain_test
    del domain_train
    del title_test
    del title_train
    del ad_train
    del ad_test
    
    return super_train, y_train, super_test, y_test, y_train_alt, y_test_alt

In [5]:
tracker = './data/tracker.csv'
data_loc = './data'
test_path = reddit_news_fech(data_loc, tracker)

We good to Go!
Done feching data
Writing data to .csv
Job Done


In [6]:
print test_path

./data/3-10_16.csv


In [ ]:
#test_path = './data/3-5_15.csv'

In [ ]:
meta_path = './data/meta_data.csv'
meta_data = pd.read_csv(meta_path)

cv = KFold(n_splits=3, shuffle=True)
mm = MinMaxScaler()
mlp = MLPClassifier()

mlp_pipe = Pipeline(steps=[('mm', mm), ('mlp', mlp)])

mlp_params = {'mlp__hidden_layer_sizes': [tuple(randint(50, 500).rvs(m)) for m in range(1, 6)],
              'mlp__activation': ['identity', 'logistic', 'tanh', 'relu'],
              'mlp__solver': ['lbfgs', 'sgd', 'adam'],
              'mlp__learning_rate': ['constant', 'invscaling', 'adaptive'],
              'mlp__alpha': np.logspace(-7, 1, 10**4),
              'mlp__max_iter': randint(1000, 5000).rvs(1000)}

ran_mlp = RandomizedSearchCV(mlp_pipe, param_distributions=mlp_params,
                             cv=cv, n_iter=33, n_jobs=-1, verbose=2)

train_set = pd.read_csv('./data/TRAIN.csv', na_filter=False)
test_set = pd.read_csv(test_path, na_filter=False)
    
test_mean = test_set['upvotes'].mean()

print train_set.shape
print test_set.shape
    
X_train, y_train, X_test, y_test, y_train_mean, y_test_mean = reddit_data_process(train_set, test_set)
    
baseline = float(y_test.sum())/len(y_test)
if baseline < .5:
        baseline = 1 - float(y_test.sum())/len(y_test)
alt_baseline = 1 - float(sum(y_test_mean))/len(y_test_mean)
    
print '\nBaseline: {}\nAlt_Baseline: {}\n'.format(baseline,
                                                  alt_baseline)

ran_mlp.fit(X_train, y_train)
net_score = ran_mlp.score(X_test, y_test)
net_percent = (net_score - baseline) * 100
net_best_params = ran_mlp.best_params_
    
print '\n{}: {}\nFrom Baseline: {}%\n{}\n'.format('Median', net_score,
                                                  net_percent,
                                                  net_best_params)
    
ran_mlp.fit(X_train, y_train_mean)
net_score_mean = ran_mlp.score(X_test, y_test_mean)
net_percent_mean = (net_score_mean - alt_baseline) * 100
net_best_params_mean = ran_mlp.best_params_

print '\n{}: {}\nFrom Baseline: {}%\n{}\n'.format('Mean', net_score_mean,
                                                  net_percent_mean,
                                                  net_best_params_mean)
    
result = pd.DataFrame([[tracker['time'].iloc[-1], len(X_train), len(X_train.columns),
                        tracker['median'].iloc[-1], baseline, net_score, net_best_params,
                        test_mean, alt_baseline, net_score_mean, net_best_params_mean]],
                      columns=['time', 'samples', 'features',
                               'median', 'baseline', 'score', 'params',
                               'mean', 'mean_base', 'mean_score', 'mean_params'])

meta_data = meta_data.append(result)
train_set = train_set.append(test_set)

print 'Writing to .cvs now...'
train_set.to_csv('./data/TRAIN.csv', index=False)

tr = pd.read_csv('./data/tracker_mean.csv')

tr.append(pd.DataFrame([[tracker['time'].iloc[-1], tracker['median'].iloc[-1], test_mean]],
                       columns=['time', 'median', 'mean']))

tr.to_csv('./data/tracker_mean.csv', index=False)

meta_data.to_csv(meta_path, index=False)
print 'All Done'

In [ ]:
sum(net_best_params_mean['mlp__hidden_layer_sizes'])

In [ ]:
len(net_best_params_mean['mlp__hidden_layer_sizes'])

In [ ]:
meta_path = './data/meta_data.csv'
meta_data = pd.read_csv(meta_path)
meta_data.head(3)

In [ ]:
train = pd.read_csv('./data/TRAIN.csv', na_filter=False)
train.shape

In [ ]:
train_path = './data/TRAIN.csv'
train = pd.read_csv('./data/3-1_19.csv', na_filter=False)

dates = ['3-2_16', '3-3_2', '3-3_15', '3-4_5', '3-4_16',
         '3-5_3', '3-5_15', '3-6_2', '3-6_14', '3-7_2', '3-7_16', '3-8_3', '3-8_17', '3-9_4', '3-9_16', '3-9_22', '3-10_16']

mean_list = [train['upvotes'].mean()]

track_path = './data/tracker.csv'
tracker = pd.read_csv(track_path)

meta_path = './data/meta_data.csv'
meta_columns = ['time', 'samples', 'features', 'test_size',
                'median', 'baseline', 'score', 'log_score',
                'nodes', 'hidden_layers', 'complexity', 'alpha', 'solver', 'activation', 'max_iter',
                'mean', 'mean_base', 'mean_score', 'mean_log_score',
                'nodes_mean', 'hidden_layers_mean', 'complexity_mean', 'alpha_mean', 'solver_mean',
                'activation_mean', 'max_iter_mean']
meta_data = pd.DataFrame(columns=meta_columns)

cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
mm = MinMaxScaler()
mlp = MLPClassifier()
log = LogisticRegression()

mlp_pipe = Pipeline(steps=[('mm', mm), ('mlp', mlp)])
log_pipe = Pipeline(steps=[('mm', mm), ('log', log)])

mlp_params = {'mlp__hidden_layer_sizes': [tuple(randint(50, 500).rvs(m)) for m in range(1, 6)],
              'mlp__activation': ['identity', 'logistic', 'tanh', 'relu'],
              'mlp__solver': ['lbfgs', 'sgd', 'adam'],
              'mlp__learning_rate': ['constant', 'invscaling', 'adaptive'],
              'mlp__alpha': np.logspace(-7, 3, 1000),
              'mlp__max_iter': randint(500, 5000).rvs(1000)}

log_params = {'log__C': np.logspace(-7, 3, 1000),
              'log__solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'log__class_weight':[None, 'balanced'], 
              'log__max_iter': randint(500, 5000).rvs(1000)}

ran_mlp = RandomizedSearchCV(mlp_pipe, param_distributions=mlp_params,
                             cv=cv, n_iter=25, n_jobs=-1, verbose=2)
ran_log = RandomizedSearchCV(log_pipe, param_distributions=log_params,
                             cv=cv, n_iter=25, n_jobs=-1, verbose=1)

c = 1
for d in dates:
    
    test = pd.read_csv('./data/{}.csv'.format(d), na_filter=False)
    
    test_mean = test['upvotes'].mean()
    mean_list.append(test_mean)
    
    X_train, y_train, X_test, y_test, y_train_mean, y_test_mean = reddit_data_process(train,
                                                                                      test)
    baseline = float(y_test.sum())/len(y_test)
    if baseline < .5:
        baseline = 1 - float(y_test.sum())/len(y_test)
    alt_baseline = 1 - float(sum(y_test_mean))/len(y_test_mean)
    
    print '\n{}\nMedian Baseline: {}\nMean Baseline: {}\n'.format(d,
                                                                  baseline,
                                                                  alt_baseline)
    ran_mlp.fit(X_train, y_train)
    net_score = ran_mlp.score(X_test, y_test)
    net_percent = (net_score - baseline) * 100
    net_best_params = ran_mlp.best_params_
    net_alpha = net_best_params['mlp__alpha']
    net_solver = net_best_params['mlp__solver']
    net_activation = net_best_params['mlp__activation']
    net_max_iter = net_best_params['mlp__max_iter']
    total_node = sum(net_best_params['mlp__hidden_layer_sizes'])
    depth = len(net_best_params['mlp__hidden_layer_sizes'])
    power_node = total_node * depth
    
    ran_log.fit(X_train, y_train)
    log_score = ran_log.score(X_test, y_test)
    net_log = (net_score - log_score) * 100
    if net_log > 0:
        log_p = ''
    else:
        log_p = 'LOG Params:{}'.format(ran_log.best_params_)
    
    print '\nScore[median]: {}\nFrom Baseline: {}%\nFrom LOG: {}%\n{}\n{}'.format(net_score,
                                                                                  net_percent,
                                                                                  net_log,
                                                                                  net_best_params,
                                                                                  log_p)
    ran_mlp.fit(X_train, y_train_mean)
    net_score_mean = ran_mlp.score(X_test, y_test_mean)
    net_percent = (net_score_mean - alt_baseline) * 100
    net_best_params_mean = ran_mlp.best_params_
    net_alpha_mean = net_best_params_mean['mlp__alpha']
    net_solver_mean = net_best_params_mean['mlp__solver']
    net_activation_mean = net_best_params_mean['mlp__activation']
    net_max_iter_mean = net_best_params_mean['mlp__max_iter']
    total_node_mean = sum(net_best_params_mean['mlp__hidden_layer_sizes'])
    depth_mean = len(net_best_params_mean['mlp__hidden_layer_sizes'])
    power_node_mean = total_node_mean * depth_mean
    
    ran_log.fit(X_train, y_train_mean)
    log_score_mean = ran_log.score(X_test, y_test_mean)
    net_log = (net_score_mean - log_score_mean) * 100
    if net_log > 0:
        log_p = ''
    else:
        log_p = 'LOG Params:{}'.format(ran_log.best_params_)

    print '\nScore[mean]: {}\nFrom Baseline: {}%\nFrom LOG: {}%\n{}\n{}'.format(net_score_mean,
                                                                                net_percent,
                                                                                net_log,
                                                                                net_best_params_mean,
                                                                                log_p)
    
    result = pd.DataFrame([[tracker['time'][c], len(X_train), len(X_train.columns), len(X_test),
                            tracker['median'][c], baseline, net_score, log_score,
                            total_node, depth, power_node, net_alpha, net_solver, net_activation, net_max_iter,
                            test_mean, alt_baseline, net_score_mean, log_score_mean,
                            total_node_mean, depth_mean, power_node_mean, net_alpha_mean, net_solver_mean,
                            net_activation_mean, net_max_iter_mean]],
                         columns=meta_columns)
    
    meta_data = meta_data.append(result, ignore_index=True)
    train = train.append(test, ignore_index=True)
    c = c + 1

print 'Writing to .csv now...'
train.to_csv(train_path, index=False)

tracker['mean'] = mean_list
tracker.to_csv('./data/tracker_mean.csv', index=False)

meta_data.to_csv(meta_path, index=False)
print 'All Done'


3-2_16
Median Baseline: 0.5
Mean Baseline: 0.9

Fitting 2 folds for each of 25 candidates, totalling 50 fits
[CV] mlp__activation=logistic, mlp__alpha=18.124175473742394, mlp__hidden_layer_sizes=(126, 397, 56, 315, 256), mlp__solver=lbfgs, mlp__learning_rate=invscaling, mlp__max_iter=1332 
[CV] mlp__activation=logistic, mlp__alpha=18.124175473742394, mlp__hidden_layer_sizes=(126, 397, 56, 315, 256), mlp__solver=lbfgs, mlp__learning_rate=invscaling, mlp__max_iter=1332 
[CV] mlp__activation=identity, mlp__alpha=954.9485639791966, mlp__hidden_layer_sizes=(54, 182, 222, 442), mlp__solver=sgd, mlp__learning_rate=invscaling, mlp__max_iter=1247 
[CV] mlp__activation=identity, mlp__alpha=954.9485639791966, mlp__hidden_layer_sizes=(54, 182, 222, 442), mlp__solver=sgd, mlp__learning_rate=invscaling, mlp__max_iter=1247 
[CV]  mlp__activation=logistic, mlp__alpha=18.124175473742394, mlp__hidden_layer_sizes=(126, 397, 56, 315, 256), mlp__solver=lbfgs, mlp__learning_rate=invscaling, mlp__max_iter=1

/home/loki/.local/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1247) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  mlp__activation=identity, mlp__alpha=954.9485639791966, mlp__hidden_layer_sizes=(54, 182, 222, 442), mlp__solver=sgd, mlp__learning_rate=invscaling, mlp__max_iter=1247, total=  59.9s
[CV] mlp__activation=logistic, mlp__alpha=0.4641588833612782, mlp__hidden_layer_sizes=(244, 159, 198), mlp__solver=adam, mlp__learning_rate=invscaling, mlp__max_iter=3727 


/home/loki/.local/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1247) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  mlp__activation=identity, mlp__alpha=954.9485639791966, mlp__hidden_layer_sizes=(54, 182, 222, 442), mlp__solver=sgd, mlp__learning_rate=invscaling, mlp__max_iter=1247, total= 1.0min
[CV] mlp__activation=logistic, mlp__alpha=0.4641588833612782, mlp__hidden_layer_sizes=(244, 159, 198), mlp__solver=adam, mlp__learning_rate=invscaling, mlp__max_iter=3727 
[CV]  mlp__activation=logistic, mlp__alpha=0.4641588833612782, mlp__hidden_layer_sizes=(244, 159, 198), mlp__solver=adam, mlp__learning_rate=invscaling, mlp__max_iter=3727, total=   8.4s
[CV] mlp__activation=relu, mlp__alpha=911.9267598459298, mlp__hidden_layer_sizes=(451,), mlp__solver=sgd, mlp__learning_rate=constant, mlp__max_iter=3399 
[CV]  mlp__activation=relu, mlp__alpha=911.9267598459298, mlp__hidden_layer_sizes=(451,), mlp__solver=sgd, mlp__learning_rate=constant, mlp__max_iter=3399, total=   1.7s
[CV] mlp__activation=relu, mlp__alpha=911.9267598459298, mlp__hidden_layer_sizes=(451,), mlp__solver=sgd, mlp__learning_rate=co

...

[CV]  mlp__activation=logistic, mlp__alpha=3.4477640547344643, mlp__hidden_layer_sizes=(126, 397, 56, 315, 256), mlp__solver=sgd, mlp__learning_rate=adaptive, mlp__max_iter=723, total= 9.8min
[CV] mlp__activation=relu, mlp__alpha=2.2250887981283696, mlp__hidden_layer_sizes=(244, 159, 198), mlp__solver=adam, mlp__learning_rate=invscaling, mlp__max_iter=1332 


/home/loki/.local/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (723) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[CV]  mlp__activation=logistic, mlp__alpha=3.4477640547344643, mlp__hidden_layer_sizes=(126, 397, 56, 315, 256), mlp__solver=sgd, mlp__learning_rate=adaptive, mlp__max_iter=723, total= 9.7min
[CV] mlp__activation=relu, mlp__alpha=2.2250887981283696, mlp__hidden_layer_sizes=(244, 159, 198), mlp__solver=adam, mlp__learning_rate=invscaling, mlp__max_iter=1332 
[CV]  mlp__activation=relu, mlp__alpha=2.2250887981283696, mlp__hidden_layer_sizes=(244, 159, 198), mlp__solver=adam, mlp__learning_rate=invscaling, mlp__max_iter=1332, total= 1.9min
[CV] mlp__activation=relu, mlp__alpha=14.065272421052393, mlp__hidden_layer_sizes=(126, 397, 56, 315, 256), mlp__solver=sgd, mlp__learning_rate=adaptive, mlp__max_iter=3661 
[CV]  mlp__activation=identity, mlp__alpha=97.49649183484097, mlp__hidden_layer_sizes=(126, 397, 56, 315, 256), mlp__solver=sgd, mlp__learning_rate=adaptive, mlp__max_iter=1653, total= 5.0min
[CV] mlp__activation=relu, mlp__alpha=14.065272421052393, mlp__hidden_layer_sizes=(126, 397

[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 46.5min


[CV]  mlp__activation=tanh, mlp__alpha=794.145171902934, mlp__hidden_layer_sizes=(244, 159, 198), mlp__solver=lbfgs, mlp__learning_rate=adaptive, mlp__max_iter=3139, total=  18.2s
[CV] mlp__activation=tanh, mlp__alpha=794.145171902934, mlp__hidden_layer_sizes=(244, 159, 198), mlp__solver=lbfgs, mlp__learning_rate=adaptive, mlp__max_iter=3139 
[CV]  mlp__activation=tanh, mlp__alpha=794.145171902934, mlp__hidden_layer_sizes=(244, 159, 198), mlp__solver=lbfgs, mlp__learning_rate=adaptive, mlp__max_iter=3139, total=  20.3s
[CV] mlp__activation=logistic, mlp__alpha=3.002461709085549, mlp__hidden_layer_sizes=(54, 182, 222, 442), mlp__solver=adam, mlp__learning_rate=invscaling, mlp__max_iter=3977 
[CV]  mlp__activation=relu, mlp__alpha=1.5211855179861049e-05, mlp__hidden_layer_sizes=(279, 231), mlp__solver=adam, mlp__learning_rate=invscaling, mlp__max_iter=3813, total= 1.6min
[CV] mlp__activation=logistic, mlp__alpha=3.002461709085549, mlp__hidden_layer_sizes=(54, 182, 222, 442), mlp__solver=

In [ ]:
reset = pd.read_csv('./data/3-1_19.csv')
train = reset
train.to_csv('./data/TRAIN.csv', index=False)
test = pd.read_csv('./data/3-2_16.csv', na_filter=False)

In [ ]:
test_path = './data/3-4_16.csv'
test = pd.read_csv(test_path, na_filter=False)
test.shape

In [ ]:
sum([1, 3, 4])

In [ ]:
nets = [tuple(randint(100, 1000).rvs(m)) for m in range(1, 11)]
print nets 

In [ ]:
cv = KFold(n_splits=3, shuffle=True)
mm = MinMaxScaler()
mlp = MLPClassifier()

mlp_pipe = Pipeline(steps=[('mm', mm), ('mlp', mlp)])

mlp_params = {'mlp__hidden_layer_sizes':[tuple(randint(50, 500).rvs(m)) for m in range(1, 6)],
              'mlp__activation':['identity', 'logistic', 'tanh', 'relu'], 'mlp__solver':['lbfgs', 'sgd', 'adam'],
              'mlp__alpha':np.logspace(-7, 7, 10**7), 'mlp__max_iter':randint(1000, 5000).rvs(1000)}

ran_mlp = RandomizedSearchCV(mlp_pipe, param_distributions=mlp_params, cv=cv, n_iter=10, n_jobs=-1, verbose=2)

X_train, y_train, X_test, y_test = reddit_data_process(train, test)

ran_mlp.fit(X_train, y_train)
net_score = ran_mlp.score(X_test, y_test)

print '\n{}: {}\n{}\n'.format('NNET', net_score, ran_mlp.best_params_)

In [ ]:
print 'Writing new TRAIN to .csv'
df_TRAIN = train.append(test)
df_TRAIN.to_csv('./data/TRAIN.csv', index=False)
del df_TRAIN

In [ ]:
10**0

In [ ]:
tr = pd.read_csv('./data/tracker.csv')
tr.head(3)

In [ ]:
meta_data.to_csv(meta_path, index=False)

In [ ]:
meta_data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

plt.style.use('fivethirtyeight')

In [ ]:
meta_data['params'][3]

In [ ]:
train_set.shape

In [ ]:
sns.boxplot(y=train_set['upvotes'])
plt.title('A Distribution of the TOP 10% of articles in r/news')
plt.xlabel('Articles in r/news')
plt.ylabel('Up Votes')

In [ ]:
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14
meta_data.plot(x='time', y='samples', legend=False)
plt.title('Feature Space')
plt.xlabel('Days in March')
plt.ylabel('# of Features')

In [ ]:
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14
meta_data.plot(x='time', y=['median', 'mean'], legend=True)
plt.title('r/news Engagement')
plt.xlabel('Days in March')
plt.ylabel('# of UpVotes')

In [ ]:
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

plt.plot(meta_data['time'], meta_data['score'], label='Neural Net: Median')
plt.plot(meta_data['time'], meta_data['baseline'], label='Median Baseline')
plt.plot(meta_data['time'], meta_data['mean_score'], label='Neural Net: Mean')
plt.plot(meta_data['time'], meta_data['mean_base'], label='Mean Baseline')
plt.title('Model Performance Over Data')
plt.xlabel('Time')
plt.ylabel('Accuracy Score')
plt.legend(loc='lower right')
plt.show()

In [ ]:
meta_data['best_params']

In [ ]:
cv_a = CountVectorizer(lowercase=False, token_pattern='\S+')
cv_d = CountVectorizer(token_pattern='\S+')
cv_t = CountVectorizer(stop_words='english', preprocessor=no_num)

cv = KFold(n_splits=3, shuffle=True)
mm = MinMaxScaler()

log = LogisticRegression()
knn = KNeighborsClassifier()
rfrst = RandomForestClassifier()
mb = MultinomialNB()
svm = LinearSVC()
sgd = SGDClassifier(penalty='elasticnet')
ada = AdaBoostClassifier(base_estimator=MultinomialNB(), n_estimators=2000)
gb = GradientBoostingClassifier(n_estimators=2000)


log_pipe = Pipeline(steps=[('mm', mm), ('log', log)])
knn_pipe = Pipeline(steps=[('mm', mm), ('knn', knn)])
rfrst_pipe = Pipeline(steps=[('mm', mm), ('rfrst', rfrst)])
mb_pipe = Pipeline(steps=[('mm', mm), ('mb', mb)])
svm_pipe = Pipeline(steps=[('mm', mm), ('svm', svm)])
sgd_pipe = Pipeline(steps=[('mm', mm), ('sgd', sgd)])
ada_pipe = Pipeline(steps=[('mm', mm), ('ada', ada)])
gb_pipe = Pipeline(steps=[('mm', mm), ('gb', gb)])


log_params = {'log__C': np.logspace(-7, 7, 10**7), 'log__penalty': ['l1', 'l2']}

knn_params = {'knn__n_neighbors': randint(1, 100).rvs(10**3), 'knn__weights': ['uniform', 'distance'], 'knn__p': [1, 2]}

rfrst_params = {'rfrst__criterion':['entropy', 'gini'], 'rfrst__max_features':[None, 'sqrt', 'log2'],
                'rfrst__n_estimators':randint(1, 1000).rvs(100)}

ada_params = {'ada__learning_rate':np.logspace(-7, 0, 10**4)}

gb_params = {'gb__loss':['deviance', 'exponential'], 'gb__learning_rate':np.logspace(-7, 0, 10**4),
             'gb__max_features':[None, 'sqrt', 'log2']}

mb_params = {'mb__alpha': np.logspace(-7, 7, 10**7)}

svm_params = {'svm__C': np.logspace(-7, 7, 10**7), 'svm__loss': ['hinge', 'squared_hinge']}

sgd_params = {'sgd__loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
              'sgd__alpha':np.logspace(-7, 7, 10**7), 'sgd__l1_ratio': np.linspace(0, 1, 10**7)}

ran_log = RandomizedSearchCV(log_pipe, param_distributions=log_params, cv=cv, n_iter=10**3, n_jobs=-1, verbose=1)
ran_knn = RandomizedSearchCV(knn_pipe, param_distributions=knn_params, cv=cv, n_iter=10**3, n_jobs=-1, verbose=1)
ran_rfrst = RandomizedSearchCV(rfrst_pipe, param_distributions=rfrst_params, cv=cv, n_iter=100, n_jobs=-1, verbose=1)
ran_mb = RandomizedSearchCV(mb_pipe, param_distributions=mb_params, cv=cv,  n_iter=10**5, n_jobs=-1, verbose=1)
ran_svm = RandomizedSearchCV(svm_pipe, param_distributions=svm_params, cv=cv, n_iter=10**3, n_jobs=-1, verbose=1)
ran_sgd = RandomizedSearchCV(sgd_pipe, param_distributions=sgd_params, cv=cv, n_iter=10**3, n_jobs=-1, verbose=1)
ran_ada = RandomizedSearchCV(ada_pipe, param_distributions=ada_params, cv=cv, n_iter=100, n_jobs=-1, verbose=1)
ran_gb = RandomizedSearchCV(gb_pipe, param_distributions=gb_params, cv=cv,  n_iter=100, n_jobs=-1, verbose=1)

pipes = [('LOG', ran_log), ('KNN', ran_knn), ('RFRST', ran_rfrst), ('SVM', ran_svm),
         ('SGD', ran_sgd), ('MB', ran_mb), ('ADA', ran_ada), ('GB', ran_gb)]

y_train = train['target']
y_test = test['target']

# The author is sometimes deleated, so i need to account for that
author_train = ['REDACTED' if x == None else x if isinstance(x, basestring) == True else x.name for x in train['author']]
author_test = ['REDACTED' if x == None else x if isinstance(x, basestring) == True else x.name for x in test['author']]
cv_a.fit(author_train)
author_train = pd.DataFrame(cv_a.transform(author_train).todense(), columns=cv_a.get_feature_names())
author_test = pd.DataFrame(cv_a.transform(author_test).todense(), columns=cv_a.get_feature_names())

cv_d.fit(train['domain'])
domain_train = pd.DataFrame(cv_d.transform(train['domain']).todense(), columns=cv_d.get_feature_names())
domain_test = pd.DataFrame(cv_d.transform(test['domain']).todense(), columns=cv_d.get_feature_names())

cv_t.fit(train['title'])
title_train = pd.DataFrame(cv_t.transform(train['title']).todense(), columns=cv_t.get_feature_names())
title_test = pd.DataFrame(cv_t.transform(test['title']).todense(), columns=cv_t.get_feature_names())

ad_train = author_train.merge(domain_train, left_index=True, right_index=True)
super_train = title_train.merge(ad_train, left_index=True, right_index=True)
ad_test = author_test.merge(domain_test, left_index=True, right_index=True)
super_test = title_test.merge(ad_test, left_index=True, right_index=True)

super_train['up_seconds'] = train['up_seconds'].values
title_train['up_seconds'] = train['up_seconds'].values
domain_train['up_seconds'] = train['up_seconds'].values
author_train['up_seconds'] = train['up_seconds'].values

super_test['up_seconds'] = test['up_seconds'].values
title_test['up_seconds'] = test['up_seconds'].values
domain_test['up_seconds'] = test['up_seconds'].values
author_test['up_seconds'] = test['up_seconds'].values

del ad_test
del cv_a
del cv_d
del cv_t

sets = [('SUPER', super_test, super_train), ('Title', title_test, title_train),
        ('Domain', domain_test, domain_train), ('Author', author_test, author_train)]
spc = '------------------------------------------'
for set_name, X_test, X_train in sets:
    
    print '{}\n{} set\n{}'.format(spc, set_name, spc)
    
    for name, pipe in pipes:
        print '{}\nStarting {}...'.format(spc, name)
        pipe.fit(X_train, y_train)
        mod_score = pipe.score(X_test, y_test)
        print '\n{}: {}\n{}\n{}'.format(name, mod_score, pipe.best_params_, spc)